# Get Dataset

In [1]:
# Get roboflow data for pickleball court keypoints detection training
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="K5y7hyQBmDKQ82NzgM5Q")
project = rf.workspace("pickleball-ball-detection").project("pickleball-court-keypoints-syncz")
version = project.version(6)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Pickleball-Court-Keypoints-6 in coco:: 100%|██████████| 49/49 [00:00<00:00, 2191.29it/s]


# Start Code

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [4]:
class keyPointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astytpe(np.float32)

        # Adjust keypoints
        kps[::2] *= (224 / w) # Adjust x coordinates
        kps[1::2] *= (224 / h) # Adjust y coordinates

        return img, kps
    

In [5]:
# Using RoboFlow JSON COCO format
# Adjust file paths
from pathlib import Path

root = Path(dataset.location)
inner = root / "Pickleball-1" if (root / "Pickleball-1").exists() else root  # handle your earlier nesting

train_images = inner / "train"
val_images   = inner / ("valid" if (inner/"valid").exists() else "val")

# common Roboflow layout:
train_ann = train_images / "_annotations.coco.json"
val_ann   = val_images   / "_annotations.coco.json"

# fallbacks if your export uses a central annotations dir or different names
if not train_ann.exists():
    alt = inner / "annotations" / "data_train.json"
    if alt.exists():
        train_ann = alt
if not val_ann.exists():
    alt = inner / "annotations" / "data_val.json"
    if alt.exists():
        val_ann = alt

print(train_images, train_ann)
print(val_images, val_ann)


/Users/darensivam/Desktop/Projects/pickleball/training/Pickleball-Court-Keypoints-6/train /Users/darensivam/Desktop/Projects/pickleball/training/Pickleball-Court-Keypoints-6/train/_annotations.coco.json
/Users/darensivam/Desktop/Projects/pickleball/training/Pickleball-Court-Keypoints-6/valid /Users/darensivam/Desktop/Projects/pickleball/training/Pickleball-Court-Keypoints-6/valid/_annotations.coco.json


In [6]:
# Obtain datasets
train_dataset = keyPointsDataset(str(train_images), str(train_ann))
val_dataset   = keyPointsDataset(str(val_images), str(val_ann))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create Model

In [7]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 12*2) # Replaces the last layer for 12 keypoints (x, y) pairs

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/darensivam/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:02<00:00, 47.5MB/s]


In [8]:
# Move model to device
model = model.to(device)

# Train Model

In [9]:
criteron = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [10]:
epochs = 20
for epoch in range(epochs):
    for i, (imgs, kps) in enumerate(train_loader):
        imgs, kps = imgs.to(device), kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criteron(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}")

KeyError: 0

In [ ]:
# Save the model
torch.save(model.state_dict(), "keypoint_model.pth")